In [1]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset,load_dataset
import os, re

# # -------------------------------------------------------------
# # 3️⃣ Load base model and tokenizer
# # -------------------------------------------------------------
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# # -------------------------------------------------------------
# # 4️⃣ Freeze everything
# # -------------------------------------------------------------
for param in model.parameters():
    param.requires_grad = False



# # -------------------------------------------------------------
# # 8️⃣ Training arguments
# # -------------------------------------------------------------
# os.environ["WANDB_DISABLED"] = "true"  # disable wandb
# training_args = TrainingArguments(
#     output_dir="./tinyllama-pharma-last4",
#     overwrite_output_dir=True,
#     num_train_epochs=2,
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=4,
#     learning_rate=1e-4,        # slightly higher since fewer params
#     fp16=True,
#     logging_steps=20,
#     save_steps=200,
#     save_total_limit=2,
#     report_to="none"
# )

# # -------------------------------------------------------------
# # 9️⃣ Trainer setup
# # -------------------------------------------------------------
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized,
#     data_collator=data_collator
# )

# # -------------------------------------------------------------
# # 🔟 Start training
# # -------------------------------------------------------------
# trainer.train()

# # -------------------------------------------------------------
# # 11️⃣ Save final model
# # -------------------------------------------------------------
# trainer.save_model("./tinyllama-pharma-last4-final")
# print("\n✅ Training completed and model saved!")


c:\Users\Roshan Acharya\Documents\Roshan\Projects\Pytorch\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # -------------------------------------------------------------
# # 5️⃣ Unfreeze last 4 transformer blocks + lm_head
# # -------------------------------------------------------------
for name, param in model.named_parameters():
    if any(f"layers.{i}." in name for i in range(20, 24)):  # last 4 layers
        param.requires_grad = True
    if "lm_head" in name:
        param.requires_grad = True

# # Verify how many parameters are trainable
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"✅ Trainable: {trainable/total*100:.2f}% of total parameters")


✅ Trainable: 13.97% of total parameters


In [3]:
#Data prep
import fitz
pages_text = []
def extract_pdf_to_list_fitz(pdf_path):
    doc = fitz.open(pdf_path)
   

    for page in doc:
        text = page.get_text("text").strip()
        if text:
            pages_text.append(text)
    
    doc.close()
    return pages_text

file_path = 'Metformin.pdf'
pdf_content_list_fitz = extract_pdf_to_list_fitz(file_path)
print(f"Total pages extracted with fitz: {len(pdf_content_list_fitz)}")




Total pages extracted with fitz: 1


In [4]:
import re
def chunk_paragraphs(txt):

    paragraph = []
    for block in txt:
        chunks=re.split(r'\n\s*\n', block)
        for chunk in chunks:
            clean=chunk.strip()
            if len(clean)>30:
                paragraph.append(clean)
    return paragraph

para=chunk_paragraphs(pages_text)

In [5]:
data=[{"text":p} for p in para]
dataset=Dataset.from_list(data)

In [6]:
if tokenizer.pad_token is None:
    tokenizer.pad_token=tokenizer.eos_token

In [13]:
def tokenize_fn(examples):
    tokens=tokenizer(examples['text'],truncation=True,padding="max_length",max_length=512)
    tokens['labels']= [ids.copy() for ids in tokens['input_ids']]

    return tokens

In [14]:
tokenized=dataset.map(tokenize_fn,batched=True,remove_columns=['text'])

Map: 100%|██████████| 4/4 [00:00<00:00, 401.45 examples/s]


In [16]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False   # IMPORTANT: causal LM, not masked LM
)

In [ ]:
os.environ["WANDB_DISABLED"] = "true"  # disable wandb
training_args = TrainingArguments(
    output_dir="./Models/tinyllama-pharma-last4",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,        # slightly higher since fewer params
    fp16=True,
    logging_steps=20,
    save_steps=200,
    save_total_limit=2,
    report_to="none"
)

In [18]:
# # 9️⃣ Trainer setup
# # -------------------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator
)


The model is already on multiple devices. Skipping the move to device specified in `args`.


In [19]:
# # 🔟 Start training
# # -------------------------------------------------------------
trainer.train()

Step,Training Loss


TrainOutput(global_step=2, training_loss=1.2983741760253906, metrics={'train_runtime': 21.5197, 'train_samples_per_second': 0.372, 'train_steps_per_second': 0.093, 'total_flos': 25424176349184.0, 'train_loss': 1.2983741760253906, 'epoch': 2.0})

In [ ]:
trainer.save_model("./Models/tinyllama-pharma-last4-final")
print("\n✅ Training completed and model saved!")



✅ Training completed and model saved!
